In [1]:
import os
os.chdir('..')

%pwd

'c:\\Users\\larakim\\Documents\\Data_train\\Satellite_image_classif\\ML-pipeline-Satellite-image-classification'

In [2]:
# MLFLOW_TRACKING_URI=https://dagshub.com/larakim/ML-pipeline-Satellite-image-classification.mlflow \
# MLFLOW_TRACKING_USERNAME=larakim \
# MLFLOW_TRACKING_PASSWORD=6f0caff147881a6dd8d7169da2dc21273f023c6c \
# python script.py

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/larakim/ML-pipeline-Satellite-image-classification.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'larakim'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '6f0caff147881a6dd8d7169da2dc21273f023c6c'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Modelevaluationconfig():
    model_path: Path
    data_path: Path
    batch_size: int
    img_size: tuple
    eval_path:Path
    root_dir:Path
    param_all: dict
    mlflow_uri: str
    

In [4]:
from img_classif.constants import *
from img_classif.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager():

    def __init__(self, config_path = CONFIG_FILE_PATH,param_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(param_path)


    def get_model_eval_conf(self)->Modelevaluationconfig:
        eval_config = Modelevaluationconfig(
            model_path = self.config.model_trainer.updated_model_path,
            data_path = self.config.model_trainer.data_path,
            batch_size = self.params.batch_size,
            img_size=self.params.img_size,
            root_dir = self.config.model_evaluation.root_dir,
            eval_path = self.config.model_evaluation.eval_path,
            param_all = self.params,
            mlflow_uri = self.config.model_evaluation.mlflow_uri)
        return eval_config

In [5]:
import tensorflow as tf
from sklearn.metrics import classification_report
import numpy as np
import mlflow
import mlflow.keras
from urllib.parse import urlparse

class Model_evaluation():

    def __init__(self,config) -> None:
        self.config = config
        create_directories([self.config.root_dir])

    def validation_generator(self):

        input_shape=eval(self.config.img_size)
        # Initialize the ImageDataGenerator (here, we're just rescaling the images)
        train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                fill_mode='nearest',
            validation_split=0.2) 

        self.ds_val = train_datagen.flow_from_directory(
            self.config.data_path,
            color_mode="rgb",
            batch_size=self.config.batch_size,
            shuffle=False,
            seed=10,
            subset='validation',
            interpolation="nearest",
            target_size=input_shape[:2],
        )
        self.labels = dict((v,k) for k,v in (self.ds_val.class_indices).items())
    
    def load_model(self):
        return tf.keras.models.load_model(self.config.model_path)
    
    def evaluation(self):
        
        self.model = self.load_model()
        predictions = self.model.predict(self.ds_val, verbose = 1)

        predicted_class_indices=np.argmax(predictions,axis=1)
        predictedLables= [self.labels[k] for k in predicted_class_indices]
        actualLables= [self.labels[k] for k in self.ds_val.classes]

        # Classification report
        self.cr = classification_report(actualLables, predictedLables, output_dict=True)
        dic = self.cr['weighted avg']
        dic['accuracy'] = self.cr['accuracy']
        self.save_report(dic)

    def save_report(self,dic):
        save_json(filename=Path(self.config.eval_path), data=dic)

    def log_into_mlflow(self):

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            mlflow.log_params(self.config.param_all)
            dic_score = self.cr['weighted avg']
            dic_score['accuracy'] = self.cr['accuracy']
            mlflow.log_metrics(dic_score
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="Xception")
            else:
                mlflow.keras.log_model(self.model, "model")


    

In [6]:
config = ConfigurationManager()
config_eval = config.get_model_eval_conf()
eval_model = Model_evaluation(config=config_eval)
eval_model.validation_generator()
eval_model.evaluation()
eval_model.log_into_mlflow()

['2024-04-08 13:40:24,515':'INFO':'common':'config\config.yaml loaded successfully']
['2024-04-08 13:40:24,519':'INFO':'common':'params.yaml loaded successfully']
['2024-04-08 13:40:24,522':'INFO':'common':'created directory at: artifacts/model_evaluation']
Found 1126 images belonging to 4 classes.
36/36 [==============================] - 43s 1s/step
